# Set FEniCS vector values with a column of a numpy array.

In [1]:
import dolfin as dl
import numpy as np
from scipy.linalg import eigh as eigh_scipy

Suppose that I want to set values to a vector. 

In [2]:
n = 5
v = dl.Vector(dl.MPI.comm_self, n)  

Something unexpected happens when I set values with a colunm of a numpy matrix. 

In [3]:
np.random.seed(0)
x = np.random.randn(n,2) 
print(x) 

[[ 1.76405235  0.40015721]
 [ 0.97873798  2.2408932 ]
 [ 1.86755799 -0.97727788]
 [ 0.95008842 -0.15135721]
 [-0.10321885  0.4105985 ]]


## 1. Wrong result

use `[:] = `

it will take the values from the matrix by rows.

In [4]:
v[:] = x[:,0]
print(x[:,0])
print(v[:])

[ 1.76405235  0.97873798  1.86755799  0.95008842 -0.10321885]
[1.76405235 0.40015721 0.97873798 2.2408932  1.86755799]


## 2. Good result

multiply by 1.

In [5]:
v[:] = x[:,0] * 1
print(x[:,0])
print(v[:])

[ 1.76405235  0.97873798  1.86755799  0.95008842 -0.10321885]
[ 1.76405235  0.97873798  1.86755799  0.95008842 -0.10321885]


## 3. Good result

use `.set_local`

it works properly.

In [6]:
v.set_local(x[:,0])
print(x[:,0])
print(v[:])

[ 1.76405235  0.97873798  1.86755799  0.95008842 -0.10321885]
[ 1.76405235  0.97873798  1.86755799  0.95008842 -0.10321885]


## 4. Wrong result 

Suppose that we get eigenvectors from `numpy.linalg.eigh`. This is a numpy array and things go wrong if we assign it to a fenics vector.

In [7]:
np.random.seed(0)
A = np.random.randn(n,n)
A = A + A.T
w, x = np.linalg.eigh(A)
print(x)

[[-0.41697748 -0.11590039  0.45001215 -0.57994628 -0.52330514]
 [-0.44538127 -0.15876114 -0.77686521 -0.38734071  0.15125488]
 [ 0.25364803  0.8226899  -0.04449218 -0.49180209  0.12245471]
 [ 0.73420742 -0.36330445 -0.27315382 -0.30446293 -0.40204311]
 [ 0.15610078 -0.3905806   0.34260319 -0.42315779  0.72569958]]


In [8]:
v[:] = x[:,0]
print(x[:,0])
print(v[:])

[-0.41697748 -0.44538127  0.25364803  0.73420742  0.15610078]
[-0.41697748 -0.11590039  0.45001215 -0.57994628 -0.52330514]


## 5. Good result

However, if we compute eigenvectors with `scipy.linalg.eigh`, it can be assigned correctly. This is confusing...

In [9]:
np.random.seed(0)
A = np.random.randn(n,n)
A = A + A.T
w, x = eigh_scipy(A)
print(x)

[[-0.41697748 -0.11590039  0.45001215 -0.57994628 -0.52330514]
 [-0.44538127 -0.15876114 -0.77686521 -0.38734071  0.15125488]
 [ 0.25364803  0.8226899  -0.04449218 -0.49180209  0.12245471]
 [ 0.73420742 -0.36330445 -0.27315382 -0.30446293 -0.40204311]
 [ 0.15610078 -0.3905806   0.34260319 -0.42315779  0.72569958]]


In [10]:
v[:] = x[:,0]
print(x[:,0])
print(v[:])

[-0.41697748 -0.44538127  0.25364803  0.73420742  0.15610078]
[-0.41697748 -0.44538127  0.25364803  0.73420742  0.15610078]
